In [1]:
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.
d:\repos\akhil\colorizer\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\repos\akhil\colorizer\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\repos\akhil\colorizer\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\repos\akhil\colorizer\venv\lib\site-packages\tensorflow\python

In [2]:
# Get images
file_names = [f'data/train/face_{i}.jpg'for i in range(1, 6)]
images = [img_to_array(load_img(file_name)) for file_name in file_names]
images = np.array(images, dtype=float)

In [3]:
lab_images = rgb2lab(1.0/255 * images)
lab_images.shape

(5, 224, 224, 3)

In [4]:
X = lab_images[:, :, :, 0]
X /= 100
X = X.reshape(len(images), 224, 224, 1,)
X.shape

(5, 224, 224, 1)

In [5]:
Y = lab_images[:, :, :, 1:]
Y /= 128
Y.shape

(5, 224, 224, 2)

In [6]:
test_file_names = [f'data/test/face_{i}.jpg'for i in range(1, 4)]
test_images = [img_to_array(load_img(file_name)) for file_name in test_file_names]
test_images = np.array(test_images, dtype=float)

test_lab_images = rgb2lab(1.0/255 * test_images)
test_X = (test_lab_images[:, :, :, 0] / 100).reshape(len(test_images), 224, 224, 1,)
test_Y = (test_lab_images[:, :, :, 1:] / 128)

In [72]:
test_file_names = [f'data/test/img_{i}.jpg'for i in range(1, 3)]
test_images = [img_to_array(load_img(file_name)) for file_name in test_file_names]
test_images = np.array(test_images, dtype=float)

test_lab_images = rgb2lab(1.0/255 * test_images)
test_X = (test_lab_images[:, :, :, 0] / 100).reshape(len(test_images), 224, 224, 1,)
test_Y = (test_lab_images[:, :, :, 1:] / 128)

In [62]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=1))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=1))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=1))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=1))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=1))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=1))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [40]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))

In [63]:
model.compile(optimizer='rmsprop', loss='mse')

In [64]:
model.fit(x=X, y=Y, batch_size=1, epochs=200)

Epoch 1/200
5/5 [==============================] - 2s 488ms/step - loss: 0.0059
Epoch 2/200
5/5 [==============================] - 2s 400ms/step - loss: 0.0048
Epoch 3/200
5/5 [==============================] - 2s 412ms/step - loss: 0.0049
Epoch 4/200
5/5 [==============================] - 2s 412ms/step - loss: 0.0046
Epoch 5/200
5/5 [==============================] - 2s 440ms/step - loss: 0.0046
Epoch 6/200
5/5 [==============================] - 2s 443ms/step - loss: 0.0044
Epoch 7/200
5/5 [==============================] - 2s 426ms/step - loss: 0.0046
Epoch 8/200
5/5 [==============================] - 2s 456ms/step - loss: 0.0045
Epoch 9/200
5/5 [==============================] - 2s 483ms/step - loss: 0.0045
Epoch 10/200
5/5 [==============================] - 2s 474ms/step - loss: 0.0046
Epoch 11/200
5/5 [==============================] - 2s 449ms/step - loss: 0.0045
Epoch 12/200
5/5 [==============================] - 2s 431ms/step - loss: 0.0045
Epoch 13/200
5/5 [===================

5/5 [==============================] - 3s 591ms/step - loss: 0.0029
Epoch 103/200
5/5 [==============================] - 3s 573ms/step - loss: 0.0033
Epoch 104/200
5/5 [==============================] - 3s 606ms/step - loss: 0.0032
Epoch 105/200
5/5 [==============================] - 3s 551ms/step - loss: 0.0031
Epoch 106/200
5/5 [==============================] - 3s 586ms/step - loss: 0.0031
Epoch 107/200
5/5 [==============================] - 3s 593ms/step - loss: 0.0030
Epoch 108/200
5/5 [==============================] - 3s 591ms/step - loss: 0.0032
Epoch 109/200
5/5 [==============================] - 3s 592ms/step - loss: 0.0031
Epoch 110/200
5/5 [==============================] - 3s 569ms/step - loss: 0.0031
Epoch 111/200
5/5 [==============================] - 3s 591ms/step - loss: 0.0029
Epoch 112/200
5/5 [==============================] - 3s 618ms/step - loss: 0.0033
Epoch 113/200
5/5 [==============================] - 3s 602ms/step - loss: 0.0030
Epoch 114/200
5/5 [===========

In [65]:
model.evaluate(X, Y, batch_size=1)

5/5 [==============================] - 1s 167ms/step


0.002743919938802719

In [73]:
model.evaluate(test_X, test_Y, batch_size=1)

2/2 [==============================] - 0s 200ms/step


0.028678466565907

In [74]:
pred_Y = model.predict(test_X)

In [75]:
colorized_images = np.zeros((len(test_images), 224, 224, 3,))
colorized_images[:, :, :, 0] = test_X[:, :, :, 0] * 128
colorized_images[:, :, :, 1:] = pred_Y[:] * 128
colorized_images_rgb = [lab2rgb(image) for image in colorized_images]
colorized_images.shape

(2, 224, 224, 3)

In [76]:
for i in range(len(colorized_images_rgb)):
    imsave(f"results/face_{i+1}.jpg", colorized_images_rgb[i])

In [77]:
for i in range(len(colorized_images_rgb)):
    imsave(f"results/face_{i+1}_gray.jpg", rgb2gray(colorized_images_rgb[i]))

In [61]:
model.save('models/alpha_sq.h5')

In [71]:
new_model = tf.keras.models.load_model('models/alpha_sq.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
